<a href="https://colab.research.google.com/github/srijansingh53/Metal-dent-detection/blob/master/neu_train_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import cv2
import numpy as np 
from pickle import dump
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet import ResNet101
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
os.chdir('/content/drive/My Drive/metal dent detection/')
!ls

codes  dataset	label_transform.pkl  saved_models


In [0]:
''' Declaring important parameters '''
EPOCHS = 8
INIT_LR = 1e-3
BS = 16
default_image_size = tuple((64,64))
image_size = 0
directory_root = 'dataset/NEU/NEU-CLS-64'
width=256
height=256
depth=1

In [15]:
''' Loading images, train-test split '''
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # print(image.shape)
    image = cv2.resize(image, default_image_size)
    return img_to_array(image)

# Fetch images from directory
image_list, label_list = [], []

print("[INFO] Loading images ...")
root_dir = os.listdir(directory_root)

for every_folder in root_dir:
    print(f"[INFO] Processing {every_folder} ...")
    folder_image_list = os.listdir(f"{directory_root}/{every_folder}/")

    for image in folder_image_list[:]:
        image_directory = f"{directory_root}/{every_folder}/{image}"
        if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
            image_list.append(preprocess_image(image_directory))
            label_list.append(every_folder)
print("[INFO] Image loading completed")


image_size = len(image_list)
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

normalized_image_list = np.array(image_list, dtype=np.float16) / 255.0

print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(normalized_image_list, image_labels, test_size=0.2, random_state = 42)



[INFO] Loading images ...
[INFO] Processing rs ...
[INFO] Processing pa ...
[INFO] Processing gg ...
[INFO] Processing in ...
[INFO] Processing sc ...
[INFO] Processing sp ...
[INFO] Processing rp ...
[INFO] Processing ps ...
[INFO] Processing cr ...
[INFO] Image loading completed
[INFO] Spliting data to train, test


In [70]:
model = ResNet101(include_top = False, weights = 'imagenet', input_shape = (64,64,3))
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
print(model.summary())

171450368/171446536 [==============================] - 11s 0us/step
Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
_______________________

In [71]:
for i,layer in enumerate(model.layers):
    layer.trainable=False
print(model.summary())

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [72]:
# add new classifier layers
x=GlobalAveragePooling2D()(model.output)
# flat1 = Flatten()(x)
class1 = Dense(1024, activation='relu')(x)
# drop1 = Dropout(0.25)(class1)
# class2 = Dense(512, activation='relu')(drop1)
output = Dense(9, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [73]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

history = model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    validation_steps=len(x_test) // 32,
    steps_per_epoch=len(x_train) // 32,
    epochs=10, verbose=1
)


[INFO] training network...
Train on 5796 samples, validate on 1450 samples
Epoch 1/10
181/181 [==============================] - 382s 2s/step - loss: 0.9955 - acc: 0.9215 - val_loss: 5.1418 - val_acc: 0.0579
Epoch 2/10
181/181 [==============================] - 372s 2s/step - loss: 0.7125 - acc: 0.9558 - val_loss: 4.9466 - val_acc: 0.0579
Epoch 3/10
  4/181 [..............................] - ETA: 5:36 - loss: 0.7122 - acc: 0.9558

KeyboardInterrupt: ignored

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [0]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

# save the model to disk
print("[INFO] Saving model...")
model.save('saved_models/keras.h5')
print("[INFO] Model Saved...")